In [11]:
import pandas as pd

booktemp = pd.read_csv('booktemp.csv', sep=',', usecols=[1,5,11])
print(booktemp.head())



                                  title  \
0  [피치] 2022 프리즘 데일리 다이어리 (일간, 날짜형, 미니)   
1      [세트] 킹 세종 더 그레이트 한글판 + 영문판 - 전2권   
2                   블랙 쇼맨과 이름 없는 마을의 살인   
3                            안녕, 소중한 사람   
4    부지런한 사랑 (리커버) - 몸과 마음을 탐구하는 이슬아 글방   

                                         description  \
0                                                NaN   
1  세계적인 판타지 TV 드라마 시리즈 [스타트렉]의 작가, 프로듀서, 제작자인 '조 ...   
2  매년 새 작품을 선보이는 히가시노 게이고가 새로운 시리즈로 발표하는 작품의 첫 권....   
3  당신이 붙잡아야 할 소중함에 대한 모든 것. 안녕, 소중한 사람은 익숙함 뒤에 가려...   
4  일간 이슬아 수필집 &lt;나는 울 때마다 엄마 얼굴이 된다&gt;의 이슬아 작가 ...   

                          categoryName  
0                 국내도서>달력/기타>출판사 제작 상품  
1     국내도서>소설/시/희곡>판타지/환상문학>외국판타지/환상소설  
2  국내도서>소설/시/희곡>추리/미스터리소설>일본 추리/미스터리소설  
3                       국내도서>에세이>한국에세이  
4                       국내도서>에세이>한국에세이  


In [13]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import torch

# --- Configuration ---
# Path to your CSV file
CSV_FILE_PATH = 'booktemp.csv'
# Pre-trained Sentence Transformer model to use for generating embeddings
# 'all-MiniLM-L6-v2' is a good balance of performance and speed.
# You can explore other models like 'all-mpnet-base-v2' for potentially better accuracy
# but higher resource usage.
MODEL_NAME = 'all-MiniLM-L6-v2'
# Number of top results to return
TOP_N_RESULTS = 5

# --- Load Data ---
# Ensure the CSV file exists in the same directory as your script,
# or provide the full path.
try:
    booktemp = pd.read_csv(CSV_FILE_PATH, sep=',', usecols=[1, 5, 11])
    # Rename columns for easier access
    booktemp.columns = ['title', 'description', 'categoryname']
    print("Original DataFrame head:")
    print(booktemp.head())
    print("\nDataFrame info:")
    booktemp.info()
except FileNotFoundError:
    print(f"Error: The file '{CSV_FILE_PATH}' was not found.")
    print("Please make sure 'booktemp.csv' is in the same directory as your script.")
    exit()
except Exception as e:
    print(f"An error occurred while reading the CSV: {e}")
    exit()

# --- Preprocessing ---
# Handle missing descriptions or titles by filling them with empty strings.
# This prevents errors during embedding generation.
booktemp['description'] = booktemp['description'].fillna('')
booktemp['title'] = booktemp['title'].fillna('')

# Combine title and description into a single text field for better context
# when calculating similarity against a query.
booktemp['combined_text'] = booktemp['title'] + " " + booktemp['description']

# --- Model Loading ---
print(f"\nLoading Sentence Transformer model: {MODEL_NAME}...")
try:
    model = SentenceTransformer(MODEL_NAME)
    print("Model loaded successfully.")
except Exception as e:
    print(f"Error loading model '{MODEL_NAME}': {e}")
    print("Please ensure you have an internet connection or the model is cached locally.")
    print("You might need to install sentence-transformers: pip install sentence-transformers")
    exit()

# --- Generate Embeddings ---
# This is the most computationally intensive step.
# It converts all combined texts into numerical vectors.
print("\nGenerating embeddings for book titles and descriptions...")
# Use a try-except block to catch potential issues during embedding
try:
    corpus_embeddings = model.encode(booktemp['combined_text'].tolist(), convert_to_tensor=True, show_progress_bar=True)
    print(f"Embeddings generated for {len(corpus_embeddings)} books.")
except Exception as e:
    print(f"Error during embedding generation: {e}")
    exit()

# --- AI Function: Find Most Related Book ---
def find_most_related_book(query: str, top_n: int = TOP_N_RESULTS):
    """
    Finds the most related book based on the query by comparing it
    to the combined title and description of books.

    Args:
        query (str): The input query string.
        top_n (int): The number of top related books to return.

    Returns:
        pd.DataFrame: A DataFrame containing the top_n most related books
                      with their similarity scores.
    """
    if not query:
        print("Query cannot be empty.")
        return pd.DataFrame()

    print(f"\nSearching for books related to: '{query}'")

    # Encode the query into an embedding
    query_embedding = model.encode(query, convert_to_tensor=True)

    # Calculate cosine similarity between the query embedding and all book embeddings
    # util.cos_sim returns a tensor of similarity scores
    cosine_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]

    # Get the top N scores and their indices
    # torch.topk returns values (scores) and indices
    top_results = torch.topk(cosine_scores, k=top_n)

    results = []
    for score, idx in zip(top_results[0], top_results[1]):
        book_index = idx.item() # Get the actual index from the tensor
        results.append({
            'Title': booktemp.loc[book_index, 'title'],
            'Description': booktemp.loc[book_index, 'description'],
            'Category': booktemp.loc[book_index, 'categoryname'],
            'Similarity Score': score.item() # Convert tensor score to Python float
        })

    # Create a DataFrame from the results for easy viewing
    results_df = pd.DataFrame(results)
    return results_df

# --- Example Usage ---
if __name__ == "__main__":
    # Example queries
    queries = [
        "science fiction adventure in space",
        "a thrilling mystery novel",
        "cooking recipes for beginners",
        "historical drama about ancient Rome",
        "children's story about animals"
    ]

    for q in queries:
        related_books = find_most_related_book(q, top_n=3)
        if not related_books.empty:
            print("\n--- Top Related Books ---")
            print(related_books.to_string(index=False)) # Use to_string to print full DataFrame
            print("-" * 30)
        else:
            print(f"\nNo related books found for query: '{q}'")
            print("-" * 30)

    # You can also test with your own custom query
    my_custom_query = "fantasy world with magic and dragons"
    custom_results = find_most_related_book(my_custom_query, top_n=5)
    if not custom_results.empty:
        print("\n--- Custom Query Results ---")
        print(custom_results.to_string(index=False))
        print("-" * 30)
    else:
        print(f"\nNo related books found for custom query: '{my_custom_query}'")
        print("-" * 30)


/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Original DataFrame head:
                                  title  \
0  [피치] 2022 프리즘 데일리 다이어리 (일간, 날짜형, 미니)   
1      [세트] 킹 세종 더 그레이트 한글판 + 영문판 - 전2권   
2                   블랙 쇼맨과 이름 없는 마을의 살인   
3                            안녕, 소중한 사람   
4    부지런한 사랑 (리커버) - 몸과 마음을 탐구하는 이슬아 글방   

                                         description  \
0                                                NaN   
1  세계적인 판타지 TV 드라마 시리즈 [스타트렉]의 작가, 프로듀서, 제작자인 '조 ...   
2  매년 새 작품을 선보이는 히가시노 게이고가 새로운 시리즈로 발표하는 작품의 첫 권....   
3  당신이 붙잡아야 할 소중함에 대한 모든 것. 안녕, 소중한 사람은 익숙함 뒤에 가려...   
4  일간 이슬아 수필집 &lt;나는 울 때마다 엄마 얼굴이 된다&gt;의 이슬아 작가 ...   

                          categoryname  
0                 국내도서>달력/기타>출판사 제작 상품  
1     국내도서>소설/시/희곡>판타지/환상문학>외국판타지/환상소설  
2  국내도서>소설/시/희곡>추리/미스터리소설>일본 추리/미스터리소설  
3                       국내도서>에세이>한국에세이  
4                       국내도서>에세이>한국에세이  

DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1440 entries, 0 to 1439
Data columns (total 3 columns):
 #   Colum

Batches: 100%|██████████| 45/45 [02:02<00:00,  2.72s/it]

Embeddings generated for 1440 books.

Searching for books related to: 'science fiction adventure in space'

--- Top Related Books ---
                                                                    Title                                                                                                                                              Description                      Category  Similarity Score
                                                                     1984                                                                                                                                                                      국내도서>소설/시/희곡>영미소설          0.243170
                                                                     사냥꾼들 그래비티북스 GF;Gravity Fiction 시리즈 6호는 조나단 작가의 첫 장편SF 사냥꾼들이다. 종말 이후 세상을 배경으로 한 포스트 아포칼립스로, 식인종인 돌쟁이들에게 납치된 권 씨 영감의 딸을 찾아 길을 나서는 다섯 사냥꾼들의 종횡무진 모험담이 짜릿하게 펼쳐진다. 국내도서>소설/시/희곡>과학소설(SF)>한국 과학소설          0.230068
[POD] 펠로폰네소스 전쟁사 2부 : The History of the 